## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the "pax" column, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at [neobase in Github](https://github.com/alexprengere/neobase))

Bonus point: Solve this problem using pandas (instead of any other approach)


Suggestion: follow the below plan of action:

* Get familiar with the data
* Select columns of interest
* Decide what to do with NaNs

* Make processing plan
* Develop code that works with a sample

* Adjust the code to work with Big data
* Test big data approach on a sample

* Run program with big data


## 1) Get familiar with data

Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### What if we dont want to read the whole file?

Options:

* prepare the sample

* read_csv with nrows option

We create path

In [3]:
path = '/home/dsc/Data/challenge'

We call the path

In [4]:
import os

os.chdir(path)

We check the current directory

In [5]:
! pwd

/home/dsc/Data/challenge


Now, it is time to open the file. However, as the file is bery big, we should work with a small sample of it

In [6]:
b = pd.read_csv('bookings.csv.bz2', sep="^", nrows=10000)
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
8936,2013-03-19 00:00:00,1V,US,d7c56c4e46ce90b60496e58318da4843,4503da71ec8cf558b72ec91899c8c8a9,ab24520af47ec7c14767c65c7194bd59,2013-03-18 00:00:00,21701,3815,PHL,...,DOHLHRSEA,LK,D,C,2013-04-18 23:20:00,2013-04-19 16:51:17,-1,2013,3,NULL
8608,2013-03-22 00:00:00,1P,US,82a19f9ce4e958cc4418b38c283ba844,bdc302c8855b41d81976b174c049516b,552b1660ee9e0c04ea2b0c5134725811,2013-03-22 00:00:00,8512,0,DTW,...,DTWLAX,NV,L,Y,2013-06-20 08:35:00,2013-06-20 10:07:11,4,2013,3,NULL
7111,2013-03-05 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,e2169926925660415478c881b7d4f27d,2013-03-05 00:00:00,245363,0,EWR,...,BOGFLLEWR,LY,O,Y,2013-09-05 11:35:00,2013-09-05 21:28:04,1,2013,3,NULL
9174,2013-03-29 00:00:00,1A,SA,6dd7510932fd8fb13921cb74ab4ff074,b92b0b3653fa2d8d5372d73dadfde5a1,72ca4401a9c27c02ed8cd79e852a8df7,2013-03-27 00:00:00,7068,0,RUH,...,RUHJED,DG,D,C,2013-04-07 22:00:00,2013-04-07 23:48:05,1,2013,3,RUHAA2136
3315,2013-03-28 00:00:00,1S,US,010b69a47d91cc998d4a54e48e3ec923,ec0cbe41ef198cbe1b7db397d1ebdd7f,f59777d718234d85be87350d4311e9f3,2013-03-25 00:00:00,1905,0,DCA,...,MEMDCA,FD,T,Y,2013-04-25 17:30:00,2013-04-25 20:44:22,-1,2013,3,NULL


We need to see all the columns from the file

In [7]:
pd.set_option('display.max_columns',None)

We check again the sample

In [8]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
3004,2013-03-06 00:00:00,1P,US,49c4818ab7c513649e1e3915afe272a3,5edc6644e0e07e834a515aaa12e70d31,c42e50f44de101bddb1f5945d326d6fa,2013-03-06 00:00:00,13142,0,EWR,NYC,US,PVG,SHA,CN,EWR,NYC,US,EWR,NYC,US,PVG,SHA,CN,EWRPVG,NYCSHA,CNUS,1,EWRPVG,FK,P,F,2013-03-24 11:25:00,2013-03-25 14:09:38,1,2013,3,NULL
6842,2013-03-04 00:00:00,1S,AU,df45ac703160ab4a898734f9881a776a,f4f438c15d8dbbfd7062ad0fd25d03a1,9db90e0a93d36d6ace06cf663fa53606,2013-03-04 00:00:00,14810,0,MEL,MEL,AU,HKG,HKG,HK,MEL,MEL,AU,MEL,MEL,AU,HKG,HKG,HK,HKGMEL,HKGMEL,AUHK,1,MELHKG,MI,V,Y,2013-03-07 00:50:00,2013-03-07 07:20:53,2,2013,3,NULL
2090,2013-03-07 00:00:00,1V,MX,ca3cf3ff4273ffa28388c78e13f61a47,7c327131d5f3bf2507d7c1a57be2e0a8,07ee698e05907631be7a60e1b3363d7b,2013-03-07 00:00:00,4684,0,MEX,MEX,MX,CJS,CJS,MX,MEX,MEX,MX,MEX,MEX,MX,CJS,CJS,MX,CJSMEX,CJSMEX,MXMX,0,MEXCJS,KW,W,Y,2013-03-08 16:40:00,2013-03-08 18:16:51,1,2013,3,NULL
2995,2013-03-08 00:00:00,1V,US,73d2bd747515516040ec806a72ff7fb2,0ea791cd1d8b35e744dc1a3f0e2191a0,d5ae54b9ebfe778546356b482c1c9a48,2013-03-08 00:00:00,15214,0,ORD,CHI,US,DFW,DFW,US,ORD,CHI,US,DFW,DFW,US,ORD,CHI,US,DFWORD,CHIDFW,USUS,0,DFWORD,KK,M,Y,2013-04-04 17:15:00,2013-04-04 19:34:02,1,2013,3,NULL
2551,2013-03-04 00:00:00,1P,US,1de3711f152f19eb9818e4acbf5f865b,cb0c1382c04fef83e4b8a6b2ac404049,f1766a5484009ff4f7accbdbd14ce0c8,2013-03-04 00:00:00,22832,0,JFK,NYC,US,MNL,MNL,PH,JFK,NYC,US,JFK,NYC,US,TPE,TPE,TW,JFKTPE,NYCTPE,TWUS,1,JFKTPE,MS,X,Y,2013-04-25 15:00:00,2013-04-26 18:32:56,1,2013,3,NULL


Then, we check some data from this sample

In [9]:
b.describe()

,duration,distance,intl,pax,year,month
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.0
mean,20209.543000,455.563300,0.596300,0.516300,2013.0,3.0
std,42241.125937,1738.810081,0.490663,1.783943,0.0,0.0
min,59.000000,0.000000,0.000000,-25.000000,2013.0,3.0
25%,3339.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,7734.500000,0.000000,1.000000,1.000000,2013.0,3.0
75%,19077.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,488131.000000,18481.000000,1.000000,25.000000,2013.0,3.0


In [10]:
b.shape

(10000, 38)

In [11]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 38 columns):
act_date               10000 non-null object
source                 10000 non-null object
pos_ctry               10000 non-null object
pos_iata               10000 non-null object
pos_oid                10000 non-null object
rloc                   10000 non-null object
cre_date               10000 non-null object
duration               10000 non-null int64
distance               10000 non-null int64
dep_port               10000 non-null object
dep_city               10000 non-null object
dep_ctry               10000 non-null object
arr_port               10000 non-null object
arr_city               10000 non-null object
arr_ctry               10000 non-null object
lst_port               10000 non-null object
lst_city               10000 non-null object
lst_ctry               10000 non-null object
brd_port               10000 non-null object
brd_city               10000 non-null o

Clean the column names

We check the titles of the columns

In [12]:
list(b.columns) 

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

It exists whitespaces that we will work later

## 2) Select the columns of interest 

We select the necessary columns to work with the question

In [13]:
b = pd.read_csv('bookings.csv.bz2', sep="^", usecols=['arr_port','pax','year'], nrows=10000)

Another way to obtain the same result

nteresting_cols = ['pax','arr_port','year'] 

df2 = df1[interesting_cols]

df2.sample(5)

Sample

In [14]:
b.sample(5)

,arr_port,pax,year
7154,BGO,1,2013
8394,LAX,1,2013
5391,BNA,-1,2013
9100,CAN,-1,2013
3656,PVG,2,2013


## 3) What to do with NaN?



In the sample everything might be ok, but we should prepare for NaN case

In [15]:
b.count()

arr_port    10000
pax         10000
year        10000
dtype: int64

We check that there is no NaN

If we would like to solve this inconvenient

b = b.dropna() 

## 4) Make processing plan
1) get only the bookings from 2013

2) group by arr_port, sum

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

We get only the year. After that we grouby and sort from up to down

In [16]:
b2 = b [b['year'] == 2013]
b3 = b2.groupby('arr_port').agg({'pax' :'sum'}).sort_values(by='pax',ascending=False).head(10)
b3

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91
SFO,91
MCO,90
DCA,82
DEN,79


## 5) Adjust the code to work with Big data


Hint: check out https://pandas.pydata.org/pandas-docs/stable/io.html#io-chunking

#### Now we need to put together the results from all the chunks

Options:

* df.append()

* pd.concat()
    
    
pd.concat handles several DataFrames in one go, while df.append creates a new DataFrame every time we call it.

There is more than one way to do it. In this case, we are going to do by two ways

## Complete Solution 1

In [17]:
c = pd.read_csv('bookings.csv.bz2',sep='^', usecols = ['arr_port','pax','year'],nrows=10000)

In [18]:
type(c)

pandas.core.frame.DataFrame

We have read the file as pandas

However, if we read including iterator, we are using a pointer. The file is on read mode

In [19]:
c_it = pd.read_csv('bookings.csv.bz2',sep='^', usecols = ['arr_port','pax','year'],nrows=10000,iterator=True)

In [20]:
type(c_it)

pandas.io.parsers.TextFileReader

Now, we can move inside the file using getchunk

In [21]:
c_it0=c_it.get_chunk(6000)  #coge 6000 lineas del c_it y se hace dataframe

This chunk is pandas too, but not in read mode, just as file. After that we could work with for loop to read the whole file

In [22]:
type(c_it0)

pandas.core.frame.DataFrame

## Complete Solution 2

In this second way, we use chunksize from the start.

In [23]:
c_ck = pd.read_csv('bookings.csv.bz2',sep='^', usecols = ['arr_port','pax','year'],nrows=10000, chunksize=3000)

Now, we use for loop

In [24]:
for i, chunk in enumerate(c_ck):
    print(chunk.head())
    print("-------------------")

   arr_port  pax  year
0  LHR        -1  2013
1  CLT         1  2013
2  CLT         1  2013
3  SVO         1  2013
4  SVO         1  2013
-------------------
      arr_port  pax  year
3000  DEN         1  2013
3001  LAX         1  2013
3002  LAX         1  2013
3003  BUF         2  2013
3004  PVG         1  2013
-------------------
      arr_port  pax  year
6000  BOS         1  2013
6001  BOS        -1  2013
6002  BOS        -1  2013
6003  BOS         1  2013
6004  PHX         1  2013
-------------------
      arr_port  pax  year
9000  LGA        -1  2013
9001  LGA         1  2013
9002  GIG         1  2013
9003  GIG         1  2013
9004  GIG        -1  2013
-------------------


We check that it is working well.

We create the entire program

In [25]:
%%time
import pandas as pd

c_ck2 = pd.read_csv('bookings.csv.bz2',sep='^', usecols = ['arr_port','pax','year'],chunksize=2000000)

all_chunks=pd.DataFrame() #necesario crear un dataframe vacio

for i, chunk in enumerate(c_ck2):
    print(i)
    df=chunk.dropna() #ponemos df para no sobreescribir
    df=df[df['year']==2013]
    del df['year']
    small_chunk_result = df.groupby('arr_port').agg({'pax' :'sum'})
    all_chunks=all_chunks.append(small_chunk_result)
    
top_airports=all_chunks.groupby('arr_port').agg({'pax' :'sum'}).sort_values(by='pax',ascending=False)

0
1
2
3
4
5
CPU times: user 3min 31s, sys: 2.71 s, total: 3min 34s
Wall time: 3min 34s


The final solution

In [26]:
top_airports.head(10)

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


### Airport Names

Using [neobase](https://github.com/alexprengere/neobase), the successor to GeoBases. It's a referential data library that particularly focuses on airports.

We import neobase

In [27]:
from neobase import NeoBase

In [28]:
b = NeoBase()

In [29]:
b.FIELDS

(('iata_code', 0, None),
 ('name', 6, None),
 ('lat', 8, None),
 ('lng', 9, None),
 ('page_rank', 12, <function neobase.neobase.NeoBase.<lambda>>),
 ('country_code', 16, None),
 ('country_name', 18, None),
 ('continent_name', 19, None),
 ('timezone', 31, None),
 ('city_code_list', 36, <function neobase.neobase.NeoBase.<lambda>>),
 ('city_name_list', 37, <function neobase.neobase.NeoBase.<lambda>>),
 ('location_type', 41, list),
 ('currency', 46, None))

We can find the name airport of Alicante

In [30]:
b.get("ALC")['name']

'Alicante–Elche Airport'

There are many whitespaces!!! Be aware of this

In [31]:
list(top_airports.index)[0:10]

['LHR     ',
 'MCO     ',
 'LAX     ',
 'LAS     ',
 'JFK     ',
 'CDG     ',
 'BKK     ',
 'MIA     ',
 'SFO     ',
 'DXB     ']

In [32]:
top_airports.head(10)

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


Distance between two airports

In [34]:
b.distance('ORY', 'CDG')

34.87480576468853

In [35]:
round(b.distance('ORY', 'CDG'),2)

34.87

Location of ORY

In [36]:
b.get_location('ORY')

LatLng(lat=48.72528, lng=2.35944)

### Whitespaces

If we want to delete whitespaces from columns

We reset index

In [43]:
top_airports.reset_index(inplace=True)

We check

In [44]:
top_airports.head()

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0


We delete whitespaces

In [48]:
top_airports['arr_port2']=top_airports['arr_port'].map(lambda x: x.strip())

In [49]:
top_airports.head()

,arr_port,pax,arr_port2
0,LHR,88809.0,LHR
1,MCO,70930.0,MCO
2,LAX,70530.0,LAX
3,LAS,69630.0,LAS
4,JFK,66270.0,JFK


In [50]:
top_airports['arr_port2'][0]

'LHR'

Now, we should delete city airpot CPQ

We check the length before and after the remove

In [51]:
len(top_airports)

2274

In [52]:
top_airports2 = top_airports[      top_airports['arr_port2'] != 'CPQ'     ]

In [54]:
len(top_airports2)

2273

In [55]:
top_airports2['Airport_Name'] = top_airports2['arr_port'].map(lambda x: b.get(x.rstrip(), 'name'))
top_airports2.head(5)

/home/dsc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,arr_port,pax,arr_port2,Airport_Name
0,LHR,88809.0,LHR,London Heathrow Airport
1,MCO,70930.0,MCO,Orlando International Airport
2,LAX,70530.0,LAX,Los Angeles International Airport
3,LAS,69630.0,LAS,McCarran International Airport
4,JFK,66270.0,JFK,John F. Kennedy International Airport


Finally, we check the directory and save the file as csv

In [56]:
pwd

'/home/dsc/Data/challenge'

In [57]:
top_airports2.to_csv('/home/dsc/Data/challenge/top_airports3.csv', sep='^', index=False)